# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [6]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')


In [10]:
# Looking into what is included in the database
pd.read_sql("""
SELECT *
  FROM sqlite_schema;
  """,conn)

,type,name,tbl_name,rootpage,sql
0,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
1,table,offices,offices,32,"CREATE TABLE ""offices"" (\n\t""officeCode""\tINTE..."
2,table,customers,customers,57,"CREATE TABLE ""customers"" (\n\t""customerNumber""..."
3,table,employees,employees,35,"CREATE TABLE ""employees"" (\n\t""employeeNumber""..."
4,table,orderdetails,orderdetails,2,"CREATE TABLE ""orderdetails"" (\n\t""orderNumber""..."
5,table,orders,orders,27,"CREATE TABLE ""orders"" (\n\t""orderNumber""\tINTE..."
6,table,payments,payments,28,"CREATE TABLE ""payments"" (\n\t""customerNumber""\..."
7,table,products,products,4,"CREATE TABLE ""products"" (\n\t""productCode""\tTE..."


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [24]:
pd.read_sql("""
SELECT firstName, lastName
  FROM employees AS e
  JOIN offices AS o
    on e.officeCode = o.officeCode
 WHERE city = "Boston";
  """,conn).head()


,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [38]:
pd.read_sql("""
  SELECT COUNT(employeeNumber) as num_employees,
         city
    FROM employees AS e
    JOIN offices AS o
      ON e.officeCode = o.officeCode
GROUP BY o.officeCode
ORDER BY num_employees;
  """,conn)

,num_employees,city
0,2,Boston
1,2,NYC
2,2,Tokyo
3,2,London
4,4,Sydney
5,5,Paris
6,6,San Francisco


## Write 3 questions of your own and answer them

In [26]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

'\nHow many customers are there per office?\n'

In [36]:
"""
What employees were the top fives sales representatives and 
how much product did they sell?
"""
pd.read_sql("""
  SELECT e.firstName, 
         e.lastName, 
         t.total_sales
  FROM employees AS e
  JOIN (SELECT SUM(amount) AS total_sales,
               salesRepEmployeeNumber
          FROM customers AS c
          JOIN payments AS p
               USING(customerNumber)
      GROUP BY salesRepEmployeeNumber
         ORDER BY total_sales DESC) AS t
    ON t.salesRepEmployeeNumber = e.employeeNumber;
  """,conn).head()

,firstName,lastName,total_sales
0,Leslie,Jennings,989906.55
1,Leslie,Thompson,347533.03
2,Julie,Firrelli,386663.20
3,Steve,Patterson,449219.13
4,Foon Yue,Tseng,488212.67


In [42]:
"""
What is the average price of a product per product line?
"""

pd.read_sql("""
  SELECT productLine,
         AVG(priceEach) AS avg_price
    FROM products AS p
    JOIN orderdetails AS o
         USING(productCode)
GROUP BY productLine
ORDER BY avg_price DESC;
  """,conn).head()

,productLine,avg_price
0,Classic Cars,108.004475
1,Trucks and Buses,92.709253
2,Motorcycles,87.322925
3,Planes,80.327202
4,Vintage Cars,78.356088


In [54]:
"""
Which product vendors take have the longest time between order and shipment?
Which have the least?"""

pd.read_sql("""
 SELECT AVG(ship_time) as avg_ship_time,
        productVendor
   FROM products as p
   JOIN (SELECT (julianday(shippedDate)-julianday(orderDate)) as ship_time,
                productCode
           FROM orders AS o1
           JOIN orderdetails as o2
                USING(orderNumber)) AS o
         USING(productCode)
GROUP BY productVendor
ORDER BY avg_ship_time DESC
   LIMIT 1;
  """,conn)

,avg_ship_time,productVendor
0,4.262673,Highway 66 Mini Classics


In [55]:
pd.read_sql("""
 SELECT AVG(ship_time) as avg_ship_time,
        productVendor
   FROM products as p
   JOIN (SELECT (julianday(shippedDate)-julianday(orderDate)) as ship_time,
                productCode
           FROM orders AS o1
           JOIN orderdetails as o2
                USING(orderNumber)) AS o
         USING(productCode)
GROUP BY productVendor
ORDER BY avg_ship_time
   LIMIT 1;
  """,conn)

,avg_ship_time,productVendor
0,3.467811,Motor City Art Classics


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [148]:
# Here is the dataframe that techically answers the question. 
# However, I am also interested in creating a dictionary with 
# Names as keys and a list of products as values. 
# I admit this might be useless, but it'll be good practice. 
df1 = pd.read_sql("""
    SELECT e.firstName,
           e.lastName,
           p.productName
    FROM employees AS e
    LEFT JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN orders AS o2
        USING(customerNumber)
    LEFT JOIN orderDetails AS o
        USING(orderNumber)
    LEFT JOIN products AS p
        USING(productCode)
    ORDER BY e.lastName, e.firstName, p.productName;
  """,conn)

In [177]:
# Let's query to get all of the unique employee's names. 
emp_uniq = pd.read_sql("""
    SELECT lastName||", "||firstName AS full_name
    FROM employees
    GROUP BY employeeNumber
    ORDER BY full_name;
  """,conn)
emp_uniq

,full_name
0,"Bondur, Gerard"
1,"Bondur, Loui"
2,"Bott, Larry"
3,"Bow, Anthony"
4,"Castillo, Pamela"
5,"Firrelli, Jeff"
6,"Firrelli, Julie"
7,"Fixter, Andy"
8,"Gerard, Martin"
9,"Hernandez, Gerard"


In [141]:
prod_dict = {}

for i in range(0,len(emp_uniq)):
    name = emp_uniq['firstName'][i]+" "+emp_uniq['lastName'][i]
    prod_dict[name] = []

prod_dict

{'Gerard Bondur': [],
 'Loui Bondur': [],
 'Larry Bott': [],
 'Anthony Bow': [],
 'Pamela Castillo': [],
 'Jeff Firrelli': [],
 'Julie Firrelli': [],
 'Andy Fixter': [],
 'Martin Gerard': [],
 'Gerard Hernandez': [],
 'Leslie Jennings': [],
 'Barry Jones': [],
 'Yoshimi Kato': [],
 'Tom King': [],
 'Peter Marsh': [],
 'Diane Murphy': [],
 'Mami Nishi': [],
 'Mary Patterson': [],
 'Steve Patterson': [],
 'William Patterson': [],
 'Leslie Thompson': [],
 'Foon Yue Tseng': [],
 'George Vanauf': []}

In [142]:
for i in range(0,len(df1)):
    key = df1['firstName'][i]+" "+df1['lastName'][i]
    value = df1['productName'][i]
    prod_dict[key].append(value)


In [146]:
#prod_dict

## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [152]:
pd.read_sql("""
    SELECT e.firstName,
           e.lastName,
           SUM(o.quantityOrdered) AS num_sold
    FROM employees AS e
    LEFT JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN orders AS o2
        USING(customerNumber)
    LEFT JOIN orderDetails AS o
        USING(orderNumber)
    LEFT JOIN products AS p
        USING(productCode)
    GROUP BY e.employeeNumber
    ORDER BY e.lastName, e.firstName;
  """,conn)

,firstName,lastName,num_sold
0,Gerard,Bondur,NaN
1,Loui,Bondur,6186.0
2,Larry,Bott,8205.0
3,Anthony,Bow,NaN
4,Pamela,Castillo,9290.0
5,Jeff,Firrelli,NaN
6,Julie,Firrelli,4227.0
7,Andy,Fixter,6246.0
8,Martin,Gerard,4180.0
9,Gerard,Hernandez,14231.0


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [159]:
pd.read_sql("""
    SELECT e.firstName,
           e.lastName
    FROM employees AS e
    LEFT JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN orders AS o2
        USING(customerNumber)
    LEFT JOIN orderDetails AS o
        USING(orderNumber)
    LEFT JOIN products AS p
        USING(productCode)
    GROUP BY e.employeeNumber
    HAVING COUNT(*) >200
    ORDER BY e.lastName, e.firstName;
  """,conn)

,firstName,lastName
0,Larry,Bott
1,Pamela,Castillo
2,Gerard,Hernandez
3,Leslie,Jennings
4,Barry,Jones
5,George,Vanauf


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!